In [1]:
import scanpy as sc
import pytometry as pt
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
BASE = 'G:\\My Drive\\colab\\cytometry\\2022_Nature_Becher\\surface_panel\\batch_1\\'
fls = sorted(os.listdir(BASE))
fls.pop(0)
first = fls[0]

In [3]:
fpath = BASE + first
adata = pt.io.read_fcs(fpath)
sc.pp.subsample(adata, 0.01)

In [4]:
adata

AnnData object with n_obs × n_vars = 1083 × 36
    var: 'Channel Number', 'marker', '$PnD', '$PnB', '$PnR', 'channel', '$PnG', '$PnE'
    uns: 'meta'

In [5]:
sc.pp.neighbors(adata)

In [6]:
pt.tl.humap(adata)

In [7]:
adata

AnnData object with n_obs × n_vars = 1083 × 36
    var: 'Channel Number', 'marker', '$PnD', '$PnB', '$PnR', 'channel', '$PnG', '$PnE'
    uns: 'meta', 'neighbors', 'humap_settings', 'humap_scales'
    obsp: 'distances', 'connectivities'

In [8]:
%%timeit
pt.tl.humap(adata)

44.3 s ± 250 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Compiling _helper_method_AoI could be significantly enhance the speed since it's called a lot (mapped over the array)
Refer to "Function calls" section of Cython

AoI calculation seems to be a bottleneck!
Another function that should be optimized it the landmark collection function

In [11]:
%%prun -s cumulative -l _helper_
pt.tl.humap(adata)

         55387539 function calls (55294224 primitive calls) in 65.995 seconds

   Ordered by: cumulative time
   List reduced from 2683 to 6 due to restriction <'_helper_'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1083    1.121    0.001   44.999    0.042 _humap.py:269(_helper_method_AoI)
     1300    0.189    0.000   14.669    0.011 _humap.py:332(_helper_method_get_landmarks)
     1083    0.009    0.000    0.357    0.000 _humap.py:364(_helper_method_calc_T)
      217    0.002    0.000    0.066    0.000 _humap.py:243(_helper_method_T_next_mul_W)
      217    0.000    0.000    0.052    0.000 _humap.py:250(_helper_method_T_next_row_div)
       12    0.000    0.000    0.000    0.000 base.py:975(get_helper_class)

In [9]:
%%prun -s cumulative
pt.tl.humap(adata)

         55407778 function calls (55314457 primitive calls) in 65.052 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   65.054   65.054 {built-in method builtins.exec}
        1    0.000    0.000   65.054   65.054 <string>:1(<module>)
        1    0.000    0.000   65.054   65.054 _humap.py:50(humap)
   208159    0.166    0.000   46.939    0.000 _base.py:589(__mul__)
   208553    0.735    0.000   46.787    0.000 _base.py:510(_mul_dispatch)
   208159    2.492    0.000   44.835    0.000 _compressed.py:507(_mul_sparse_matrix)
        2    0.001    0.000   43.487   21.743 _construct.py:528(vstack)
        1    0.000    0.000   43.481   43.481 _humap.py:254(_calc_AoI)
        2    0.006    0.003   43.469   21.735 _construct.py:559(<listcomp>)
     1083    1.044    0.001   43.399    0.040 _humap.py:269(_helper_method_AoI)
424612/421996    2.180    0.000   22.693    0.000 _compressed.py:26(__init__)
  

In [10]:
%load_ext line_profiler

In [13]:
%lprun -f pt.tl.humap pt.tl.humap(adata)

Scale Number 0


Timer unit: 1e-07 s

Total time: 59.9951 s
File: C:\Users\ergun\pycharm\pytometry\pytometry\tools\_humap.py
Function: humap at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents
    23                                           def humap(adata, channel_idx=None, beta=100, beta_thresh=1.5, teta=50, n_scales=1, copy=False):
    24         1         19.0     19.0      0.0      if channel_idx is None:
    25         1        197.0    197.0      0.0          channel_idx = range(len(adata.var_names))
    26                                               elif len(channel_idx) == 0:
    27                                                   channel_idx = range(len(adata.var_names))
    28                                           
    29                                               # settings dict for all important setting variables
    30         1         11.0     11.0      0.0      settings = {
    31         1          7.0      7.0      0.0          'beta': beta,
    32  

In [10]:
%%lprun
pt.tl.humap(adata)

UsageError: Cell magic `%%lprun` not found.
